# first try

In [181]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
import pandas as pd
import numpy as np
import xgboost
import lightgbm as lgb
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier


# X = pd.read_csv('data/X.csv')
# y = pd.read_csv('data/y.csv')

# #train test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # used for kaggle submission
# X_kaggle = pd.read_csv('data/X_kaggle.csv')


X = pd.read_csv('data/X.csv').to_numpy()
y = pd.read_csv('data/y.csv').to_numpy().ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_kaggle = pd.read_csv('data/X_kaggle.csv').to_numpy()

### XGBoost ###

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score
import xgboost

# Load the data
X = pd.read_csv('data/X.csv')
y = pd.read_csv('data/y.csv')

# Train, validation, and test split (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)  # 60% train
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% val, 20% test

# Used for Kaggle submission
X_kaggle = pd.read_csv('data/X_kaggle.csv')

# Define the parameter grid
kf = KFold(n_splits=5)

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'random_state': [42],
    'subsample': [0.8, 1]
}

# Initialize the XGBClassifier
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV with training data
grid_search.fit(X_train, y_train)

# Best XGB model after GridSearch
best_xgb_model = grid_search.best_estimator_
print(f"Best GridSearchCV Score on Training Data: {grid_search.best_score_}")

# Evaluate on the validation set to check for overfitting
y_pred_val = best_xgb_model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_pred_val)
print(f"Accuracy on Validation Set: {val_accuracy}")

# Predict on the test set
y_pred_test = best_xgb_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy on Test Set: {test_accuracy}")

# Predict on the Kaggle dataset for submission
y_pred_xgb = best_xgb_model.predict(X_kaggle)

# Prepare submission file
y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=['prediction'])
y_pred_xgb['ID'] = y_pred_xgb.index
y_pred_xgb

# Optional: Save predictions to CSV for submission
y_pred_xgb.to_csv('y_pred_xgb.csv', index=False)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best GridSearchCV Score on Training Data: 0.9271885521885522
Accuracy on Validation Set: 0.9345959595959596
Accuracy on Test Set: 0.927020202020202


In [26]:
print(y_pred_xgb['prediction'].value_counts())

prediction
1    775
0    691
Name: count, dtype: int64


In [67]:
# Define the parameter grid
kf = KFold(n_splits=5)

param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2],
    'random_state': [42],
    'subsample': [0.8, 1]
}

# Initialize the XGBClassifier
xgb_model = xgboost.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb, cv=kf, scoring='accuracy', verbose=1)

# # Fit GridSearchCV
# grid_search.fit(X_train, y_train)
# best_xgb_model = grid_search.best_estimator_
# print(grid_search.best_score_)

# # Predict on the test set
# y_pred_test = best_xgb_model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred_test)
# print(f"Accuracy on test set: {accuracy}")

y_pred_xgb = xgb_model.predict(X_kaggle)

y_pred_xgb = pd.DataFrame(y_pred_xgb, columns=['prediction'])
y_pred_xgb['ID'] = y_pred_xgb.index
y_pred_xgb

,prediction,ID
0,0,0
1,1,1
2,1,2
3,1,3
4,0,4
...,...,...
1461,1,1461
1462,1,1462
1463,0,1463
1464,1,1464


In [68]:
print(y_pred_xgb['prediction'].value_counts())

prediction
1    791
0    675
Name: count, dtype: int64


In [69]:
y_pred_xgb.to_csv('y_pred_xgb2.csv', index=False)

### LazyClassifier ###

We can check which top 10 models are the best performing and furtherly train them ourselves with more delicate steps, meaning GridSearch and kFold :)

In [182]:
def custom_score(y_true, y_pred):
    class_1_ratio = np.mean(y_pred)
    imbalance_penalty = np.abs(class_1_ratio - 0.5)
    accuracy = np.mean(y_true == y_pred)
    # you can change 10 to any other number to adjust the penality strength
    score = accuracy - imbalance_penalty*7
    return score

acc5050 = make_scorer(custom_score)

In [183]:
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=accuracy_score)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

 97%|█████████████████████████████████████████▌ | 28/29 [01:54<00:11, 11.36s/it]

[LightGBM] [Info] Number of positive: 5280, number of negative: 5280
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017921 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2171
[LightGBM] [Info] Number of data points in the train set: 10560, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


100%|███████████████████████████████████████████| 29/29 [02:37<00:00,  5.43s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.89               0.89     0.89      0.89   
LGBMClassifier                     0.89               0.89     0.89      0.89   
KNeighborsClassifier               0.89               0.89     0.89      0.89   
ExtraTreesClassifier               0.89               0.89     0.89      0.89   
XGBClassifier                      0.89               0.89     0.89      0.89   
SVC                                0.88               0.88     0.88      0.88   
LogisticRegression                 0.88               0.88     0.88      0.88   
AdaBoostClassifier                 0.88               0.88     0.88      0.88   
CalibratedClassifierCV             0.88               0.88     0.88      0.88   
LinearSVC                          0.88               0.88     0.88      0.88   
LinearDiscriminantAnalysis  

### Random Forest ###

In [184]:
# ditch acc5050, do it again with 'accuracy'

# Define the parameter grid
# kf = KFold(n_splits=5)

param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2']
}

def custom_score(y_true, y_pred):
    class_1_ratio = np.mean(y_pred)
    imbalance_penalty = np.abs(class_1_ratio - 0.5)
    accuracy = np.mean(y_true == y_pred)
    # you can change 10 to any other number to adjust the penality strength
    return accuracy - imbalance_penalty*7

acc5050 = make_scorer(custom_score)

# Initialize the Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_rf = GridSearchCV(
    estimator=rf_model, 
    param_grid=param_grid_rf, 
    scoring=acc5050, 
    cv=5, 
    verbose=1, 
    n_jobs=-1
)

# Fit GridSearchCV
grid_search_rf.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", grid_search_rf.best_params_)
# dependent on scoring function not actually the acurracy but the custom score including imbalance penalty
print("Best (accuracy) score: ", grid_search_rf.best_score_)

y_pred_test_rf = grid_search_rf.best_estimator_.predict(X_test)
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred_test_rf)}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Best (accuracy) score:  0.7617424242424243
Accuracy on test set: 0.893560606060606


In [185]:
y_scores_rf = grid_search_rf.best_estimator_.predict_proba(X_kaggle)[:, 1]

sorted_scores_rf = np.sort(y_scores_rf)
threshold_index_rf = int(len(sorted_scores_rf) * 0.5)
threshold_rf = sorted_scores_rf[threshold_index_rf]

y_pred_thresholded_rf = (y_scores_rf >= threshold_rf).astype(int)

print(f"Class distribution after thresholding: {np.bincount(y_pred_thresholded_rf)}")

y_pred_thresholded_rf = pd.DataFrame(y_pred_thresholded_rf, columns=['predictions'])
y_pred_thresholded_rf["ID"] = y_pred_thresholded_rf.index

y_pred_rf = grid_search_rf.best_estimator_.predict(X_kaggle)
y_pred_rf = pd.DataFrame(y_pred_rf, columns=['predictions'])
y_pred_rf["ID"] = y_pred_rf.index

ValueError: X has 11 features, but RandomForestClassifier is expecting 12 features as input.

In [ ]:
y_pred_thresholded_rf.to_csv('data/thresholding_rf_y.csv', index=False)
y_pred_thresholded_rf['predictions'].value_counts()

In [ ]:
y_pred_rf.to_csv('data/y_pred_rf_new.csv', index=False)
y_pred_rf['predictions'].value_counts()

In [ ]:
f1_rf = f1_score(y_test, y_pred_test_rf)
f1_rf

### ExtraTreesClassifier ###

In [ ]:
param_grid_etc = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2']
}


# Initialize the ExtraTreeClassifier
etc_model = ExtraTreesClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_etc = GridSearchCV(estimator=etc_model, param_grid=param_grid_etc, cv=kf, scoring=acc5050, verbose=1)

# Fit GridSearchCV
grid_search_etc.fit(X_train, y_train)

best_etc_model = grid_search_etc.best_estimator_
print(grid_search_etc.best_score_)

y_pred_etc = best_etc_model.predict(X_kaggle)

y_pred_etc = pd.DataFrame(y_pred_etc, columns=['prediction'])
y_pred_etc['ID'] = y_pred_etc.index
y_pred_etc

In [ ]:
y_pred_test_etc = grid_search_etc.best_estimator_.predict(X_test)
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred_test_etc)}")

f1_etc = f1_score(y_test, y_pred_test_etc)
f1_etc

### LGBMClassifier ###

In [ ]:
param_grid_lgb = {
    'num_leaves': [25, 30, 50],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

def custom_score(y_true, y_pred):
    class_1_ratio = np.mean(y_pred)
    imbalance_penalty = np.abs(class_1_ratio - 0.5)
    accuracy = np.mean(y_true == y_pred)
    # you can change 10 to any other number to adjust the penality strength
    return accuracy - imbalance_penalty*7

acc5050 = make_scorer(custom_score)

# Initialize the LGBMClassifier
lgb_model = lgb.LGBMClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_lgb = GridSearchCV(
    estimator=lgb_model, 
    param_grid=param_grid_lgb, 
    scoring=acc5050, 
    cv=3, 
    verbose=1, 
    n_jobs=-1
)

# Fit GridSearchCV
grid_search_lgb.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters found: ", grid_search_lgb.best_params_)
# dependent on scoring function not actually the acurracy but the custom score including imbalance penalty
print("Best (accuracy) score: ", grid_search_lgb.best_score_)

y_pred_test_lgb = grid_search_lgb.best_estimator_.predict(X_test)
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred_test_lgb)}")

y_pred_lgb = grid_search_lgb.best_estimator_.predict(X_kaggle)
y_pred_lgb = pd.DataFrame(y_pred_lgb, columns=['predictions'])
y_pred_lgb["ID"] = y_pred_lgb.index
# y_pred_lgb.to_csv('data/y_pred_lgb.csv', index=False)


In [ ]:
y_scores_lgb = grid_search_lgb.best_estimator_.predict_proba(X_kaggle)[:, 1]

sorted_scores_lgm = np.sort(y_scores_lgb)
threshold_index_lgm = int(len(sorted_scores_lgm) * 0.5)
threshold_lgm = sorted_scores_lgm[threshold_index_lgm]

y_pred_thresholded_lgm = (y_scores_lgb >= threshold_lgm).astype(int)

print(f"Class distribution after thresholding: {np.bincount(y_pred_thresholded_lgm)}")

y_pred_thresholded_lgm = pd.DataFrame(y_pred_thresholded_lgm, columns=['predictions'])
y_pred_thresholded_lgm["ID"] = y_pred_thresholded_lgm.index

y_pred_lgm = grid_search_lgb.best_estimator_.predict(X_kaggle)
y_pred_lgm = pd.DataFrame(y_pred_lgm, columns=['predictions'])
y_pred_lgm["ID"] = y_pred_lgm.index

In [ ]:
y_pred_lgb['predictions'].value_counts()

#y_pred_lgb.to_csv('y_pred_lgb.csv', index=False)

In [ ]:
f1_lgm = f1_score(y_test, y_pred_test_lgb)
f1_lgm

In [ ]:
y_pred_thresholded_lgm.to_csv('data/thresholding_lgm_y.csv', index=False)

### AdaBoostClassifier ###

In [ ]:
param_grid_ada = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'n_estimators': [10, 50, 100, 500],
}


# Initialize the AdaBoostClassifier
ada_model = AdaBoostClassifier(random_state=42)

# Initialize GridSearchCV
grid_search_ada = GridSearchCV(estimator=ada_model, param_grid=param_grid_ada, cv=kf, scoring='accuracy', verbose=1)

# Fit GridSearchCV
grid_search_ada.fit(X_train, y_train)

best_ada_model = grid_search_ada.best_estimator_
print(grid_search_ada.best_score_)

y_pred_ada = best_ada_model.predict(X_kaggle)

y_pred_ada = pd.DataFrame(y_pred_ada, columns=['prediction'])
y_pred_ada['ID'] = y_pred_ada.index
y_pred_ada

In [ ]:
y_pred_test_ada = grid_search_ada.best_estimator_.predict(X_test)
print(f"Accuracy on test set: {accuracy_score(y_test, y_pred_test_ada)}")

f1_ada = f1_score(y_test, y_pred_test_ada)
f1_ada

In [ ]:
print("slay the boots down houston i'm deceased")

### Final files to submit (per model) ###

In [42]:
# XGBoost
y_pred_xgb.to_csv('data/y_pred_xgb.csv', index=False)

# Ramdom Forest
y_pred_rf.to_csv('data/y_pred_rf.csv', index=False)

# LightGBM
y_pred_lgb.to_csv('data/y_pred_lgb.csv', index=False)